In [1]:
import os

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn as nn
from torch import optim
from torch.utils.data.dataset import Dataset, TensorDataset
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import pandas as pd
import re
import string

C:\Users\Asus\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_device():
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

device = get_device()
print(f"Using device: {device}")

Using device: cpu


In [ ]:
file_path = 'data_train_1.csv'

df = pd.read_csv(file_path)
df = df[['content', 'label']]

labels_map = {
    "POS": 0,
    "NEU": 1,
    "NEG": 2
}

df['label'] = df['label'].map(labels_map)
df = df.dropna(subset=['content'])

In [4]:
def remove_emoji(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F700-\U0001F77F"  
        u"\U0001F780-\U0001F7FF"  
        u"\U0001F800-\U0001F8FF"  
        u"\U0001F900-\U0001F9FF"  
        u"\U0001FA00-\U0001FA6F"  
        u"\U0001FA70-\U0001FAFF"  
        u"\U00002702-\U000027B0"  
        u"\U000024C2-\U0001F251" 
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

def clean_text(text):
    text = text.lower() 
    text = re.sub(r'[^\w\s]', '', text)  
    text = re.sub(r'\d+', ' <num> ', text)  
    text = re.sub(r'\s+', ' ', text).strip() 
    return text

In [5]:
df_test = df.sample(n=15000, random_state=42)
df_test

,content,label
2604,Vẫn tặng bạn 5* vì nhiệt tình,0
16959,Dù có 5k nhưg mất uy tín,1
19018,Hàng đẹp dã man chị trang ạ,0
20374,Và con nào ăn thì con đấy chết,2
11124,Chất lượng tốt có nhiều quà tặng kèm,0
...,...,...
9676,Có giảm nhưng uống vào người rất mệt và buồn nôn,1
622,Đóng gói sản phẩm rất đẹp và chắc chắn Shop ph...,0
6249,Đây chỉ là góp ý và không nỡ để shop bị rate t...,1
926,"Chất vải và kiểu áo đều ok, dễ mặc",0


In [6]:
df_test['content'] = df_test['content'].apply(remove_emoji)  
df_test['content'] = df_test['content'].apply(clean_text)  

In [7]:
train_df, val_df = train_test_split(df_test, test_size=0.2, random_state=42)

In [8]:
from datasets import Dataset
from transformers import DataCollatorWithPadding

checkpoint = 'distilbert-base-multilingual-cased'
tokenizer = DistilBertTokenizer.from_pretrained(checkpoint)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def preprocess_function(examples):
    return tokenizer(examples["content"], truncation=True, padding = True, max_length = 128)

In [9]:
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

train_dataset = train_dataset.map(preprocess_function, batched=True)
val_dataset = val_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 3000/3000 [00:00<00:00, 4352.78 examples/s]


In [10]:
train_dataset = train_dataset.remove_columns(column_names='content')
train_dataset = train_dataset.remove_columns(column_names='__index_level_0__')
train_dataset = train_dataset.remove_columns(column_names='attention_mask')
val_dataset = val_dataset.remove_columns(column_names='content')
val_dataset = val_dataset.remove_columns(column_names='__index_level_0__')
val_dataset = val_dataset.remove_columns(column_names='attention_mask')

In [11]:
train_dataset

Dataset({
    features: ['label', 'input_ids'],
    num_rows: 12000
})

In [12]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=10, collate_fn=data_collator)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=10, collate_fn=data_collator)

In [13]:

path1='D:/kpw/crawlFB/Data-facebook/text'+ ".npy"
np.save(path1, arr)    # .npy extension is added if not given
print(path1)

NameError: name 'arr' is not defined

In [14]:
for batch in train_loader:
    print(batch.keys())
    break 

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size, max_length):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(0.3)
        self.fc1 = nn.Linear(hidden_size * 2, 64)
        self.fc2 = nn.Linear(64, 8)
        self.fc3 = nn.Linear(8, output_size)
        self.relu = nn.ReLU()
#         self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.embedding(x)
        batch_size = x.size(0)
        h0 = torch.zeros(2, batch_size, hidden_size).to(x.device)
        c0 = torch.zeros(2, batch_size, hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        
        out = self.relu(self.fc1(out[:, -1, :]))
        out = self.dropout(out)
        out = self.relu(self.fc2(out))
        out = self.dropout(out)
#         out = self.softmax(self.fc3(out))
        return out

In [16]:
# HYPER PARAMS

vocab_size = tokenizer.vocab_size
embedding_dim = 128
hidden_size = 64
output_size = 3
max_length = 128
num_epochs = 10
device = device

print(f"Vocabulary size: {vocab_size}")
print(f"Embedding dimension: {embedding_dim}")
print(f"Hidden size: {hidden_size}")
print(f"Output size: {output_size}")
print(f"Max length: {max_length}")
print(f"Number of epochs: {num_epochs}")

Vocabulary size: 119547
Embedding dimension: 128
Hidden size: 64
Output size: 3
Max length: 128
Number of epochs: 10


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim

def train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs=10, save_best_model_path=None, save_last_model_path=None):
    best_val_accuracy = 0.0
    best_model_state_dict = None

    for epoch in range(num_epochs):
        model.train()
        running_train_loss = 0.0
        correct_predictions = 0
        total_predictions = 0
        
        # Training loop
        for i, batch in enumerate(train_loader):
            inputs, labels = batch['input_ids'], batch['labels'] 
            
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_train_loss += loss.item()
            
            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)
        
        # Calculate average train loss and accuracy for the epoch
        train_loss = running_train_loss / len(train_loader)
        train_accuracy = correct_predictions / total_predictions
        
        # Validation loop
        val_loss, val_accuracy = evaluate(model, val_loader, criterion, device)
        
        # Save best model
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state_dict = model.state_dict()
            if save_best_model_path:
                torch.save(model.state_dict(), save_best_model_path)
        
        # Print epoch statistics
        print(f"Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Train Accuracy: {train_accuracy:.4f}, Val Accuracy: {val_accuracy:.4f}")
        
    # Save last model
    if save_last_model_path:
        torch.save(model.state_dict(), save_last_model_path)
        
    print("Training finished.")
    
    return best_model_state_dict, best_val_accuracy

# Evaluation function (for validation and testing)
def evaluate(model, loader, criterion, device):
    model.eval()
    correct_predictions = 0
    total_predictions = 0
    running_val_loss = 0.0
    
    with torch.no_grad():
        for i, batch in enumerate(loader):
            inputs, labels = batch['input_ids'], batch['labels']
            
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_val_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
#             print(predicted)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)
    
    val_loss = running_val_loss / len(loader)
    val_accuracy = correct_predictions / total_predictions
    
    return val_loss, val_accuracy

In [18]:
model = LSTMModel(vocab_size, embedding_dim, hidden_size, output_size, max_length).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs=20)

Epoch [1/20], Train Loss: 1.4773, Val Loss: 1.2081, Train Accuracy: 0.5218, Val Accuracy: 0.6367
Epoch [2/20], Train Loss: 1.4625, Val Loss: 1.2397, Train Accuracy: 0.5236, Val Accuracy: 0.6367
Epoch [3/20], Train Loss: 1.4530, Val Loss: 1.2373, Train Accuracy: 0.5275, Val Accuracy: 0.6367
Epoch [4/20], Train Loss: 1.4913, Val Loss: 1.2363, Train Accuracy: 0.5112, Val Accuracy: 0.6367
Epoch [5/20], Train Loss: 1.4727, Val Loss: 1.2401, Train Accuracy: 0.5148, Val Accuracy: 0.6367
Epoch [6/20], Train Loss: 1.4590, Val Loss: 1.0487, Train Accuracy: 0.5132, Val Accuracy: 0.6367
Epoch [7/20], Train Loss: 1.2743, Val Loss: 0.8753, Train Accuracy: 0.5845, Val Accuracy: 0.7497
Epoch [8/20], Train Loss: 1.1955, Val Loss: 0.8041, Train Accuracy: 0.6335, Val Accuracy: 0.7687
Epoch [9/20], Train Loss: 1.1642, Val Loss: 0.8199, Train Accuracy: 0.6613, Val Accuracy: 0.7737
Epoch [10/20], Train Loss: 1.1239, Val Loss: 0.8009, Train Accuracy: 0.6723, Val Accuracy: 0.7807
Epoch [11/20], Train Loss: 1.

(OrderedDict([('embedding.weight',
               tensor([[ 0.3264, -0.3869, -0.8010,  ..., -0.1576,  0.0389,  0.7377],
                       [-0.0929, -1.5505, -1.6151,  ...,  0.6304, -0.9454, -0.0206],
                       [ 0.5943,  1.6090, -1.7659,  ..., -0.2266,  0.3290, -1.3098],
                       ...,
                       [-0.1250, -1.3952, -0.1992,  ...,  0.7997,  0.2099, -1.0816],
                       [ 0.8842, -0.3242,  0.2966,  ..., -0.0655, -1.1030, -1.3702],
                       [ 0.7403,  0.0351,  0.3931,  ...,  0.9611, -0.3462,  0.2467]])),
              ('lstm.weight_ih_l0',
               tensor([[ 0.2365, -0.2981, -0.0142,  ...,  0.1053, -0.1460, -0.0732],
                       [ 0.3084, -0.0689,  0.1263,  ..., -0.1899, -0.0272,  0.0570],
                       [ 0.1843,  0.0373, -0.0904,  ...,  0.1602,  0.1119,  0.0270],
                       ...,
                       [ 0.0119, -0.4425,  0.2676,  ...,  0.2882, -0.2328,  0.0585],
                    